In [52]:
from utils import read_lines
import heapq

def parse_input(input_file):
    hall = tuple(['.'] * 11)
    lines = read_lines(input_file)
    room1 = (lines[2][3], lines[3][3])
    room2 = (lines[2][5], lines[3][5])
    room3 = (lines[2][7], lines[3][7])
    room4 = (lines[2][9], lines[3][9])
    return (hall, room1, room2, room3, room4)

costs = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}

doors = {
    'A': 2,
    'B': 4,
    'C': 6,
    'D': 8,
}

def part1(input_file):
    initial = parse_input(input_file)
    end_state = (tuple(['.'] * 11), ('A', 'A'), ('B', 'B'), ('C', 'C'), ('D', 'D'))
    hp = [(0, initial)]
    min_cost = {initial: 0}
    while hp:
        cost, state = heapq.heappop(hp)
        if cost > min_cost[state]:
            continue
        if state == end_state:
                return cost
        for n_cost, n_state in moves(cost, state):
            
            if n_cost < min_cost.get(n_state, float('inf')):
                min_cost[n_state] = n_cost
                heapq.heappush(hp, (n_cost, n_state))

def leave_room(c, room, hall):
    # return list of (cost, new_hall, new_room)
    ans = []
    door = doors[c]

    if room != (c, c) and room != ('.', c) and room != ('.', '.') and hall[door] == '.':
        if room[0] != '.':
            for i in range(door - 1, -1, -1):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[0]] * (1 + door - i)
                        new_hall = list(hall)
                        new_hall[i] = room[0]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', room[1])])
                else:
                    break
            for i in range(door + 1, 11):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[0]] * (1 + i - door)
                        new_hall = list(hall)
                        new_hall[i] = room[0]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall,  ('.', room[1])])
                else:
                    break
        if room[0] == '.' and room[1] != '.':
            for i in range(door - 1, -1, -1):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[1]] * (2 + door - i)
                        new_hall = list(hall)
                        new_hall[i] = room[1]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.') ])
                else:
                    break
            for i in range(door + 1, 11):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[1]] * (2 + i - door)
                        new_hall = list(hall)
                        new_hall[i] = room[1]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.')])
                else:
                    break
    return ans

def enter_room(i, c, hall, room):
    # return (can_enter, cost, new_hall, new_room)
    door = doors[c]
    if not (room == ('.', '.') or room == ('.', c)):
        return False, None, None, None
    if i < door:
        for j in range(i + 1, door + 1):
            if hall[j] != '.':
                return False, None, None, None
    else:
        for j in range(i - 1, door-1, -1):
            if hall[j] != '.':
                return False, None, None, None
    cost = (abs(i - door) + 1) * costs[c]
    if room == ('.', '.'):
        cost += costs[c]
        new_room = ('.', c)
    else:
        new_room = (c, c)
    new_hall = list(hall)
    new_hall[i] = '.'
    return True, cost, tuple(new_hall), new_room
    
def moves(cost, state):
    ans = []
    hall, room1, room2, room3, room4 = state
    for out_cost, new_hall, new_room1 in leave_room('A', room1, hall):
        n_state = (new_hall, new_room1, room2, room3, room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])
    
    for out_cost, new_hall, new_room2 in leave_room('B', room2, hall):
        n_state = (new_hall, room1, new_room2, room3, room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])
    
    for out_cost, new_hall, new_room3 in leave_room('C', room3, hall):
        n_state = (new_hall, room1, room2, new_room3, room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])

    for out_cost, new_hall, new_room4 in leave_room('D', room4, hall):
        n_state = (new_hall, room1, room2, room3, new_room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])
    
    for i, c in enumerate(hall):
        if c != '.':
            if c == 'A':
                room = room1
            elif c == 'B':
                room = room2
            elif c == 'C':
                room = room3
            else:
                room = room4
            can_enter, enter_cost, new_hall, new_room = enter_room(i, c, hall, room)
            if can_enter:
                n_cost = cost + enter_cost
                if c == 'A':
                    n_state = (new_hall, new_room, room2, room3, room4)
                elif c == 'B':
                    n_state = (new_hall, room1, new_room, room3, room4)
                elif c == 'C':
                    n_state = (new_hall, room1, room2, new_room, room4)
                else:
                    n_state = (new_hall, room1, room2, room3, new_room)
                ans.append([n_cost, n_state])
    return ans

In [53]:
part1('inputs/day23_test.txt')

12521

In [54]:
part1('inputs/day23.txt')

15538

In [49]:
def parse_input2(input_file):
    hall = tuple(['.'] * 11)
    lines = read_lines(input_file)
    room1 = (lines[2][3], 'D', 'D', lines[3][3])
    room2 = (lines[2][5], 'C', 'B', lines[3][5])
    room3 = (lines[2][7], 'B', 'A', lines[3][7])
    room4 = (lines[2][9], 'A', 'C', lines[3][9])
    return (hall, room1, room2, room3, room4)

costs = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000,
}

doors = {
    'A': 2,
    'B': 4,
    'C': 6,
    'D': 8,
}

def part2(input_file):
    initial = parse_input2(input_file)
    end_state = (tuple(['.'] * 11), ('A', 'A', 'A', 'A'), ('B', 'B', 'B', 'B'), ('C', 'C', 'C', 'C'), ('D', 'D', 'D', 'D'))
    hp = [(0, initial)]
    min_cost = {initial: 0}
    while hp:
        cost, state = heapq.heappop(hp)
        if cost > min_cost[state]:
            continue
        if state == end_state:
            return cost
        for n_cost, n_state in moves2(cost, state):
            
            if n_cost < min_cost.get(n_state, float('inf')):
                min_cost[n_state] = n_cost
                heapq.heappush(hp, (n_cost, n_state))

def leave_room2(c, room, hall):
    # return list of (cost, new_hall, new_room)
    ans = []
    door = doors[c]

    if room != (c, c, c, c) and room != ('.', c, c, c) and room != ('.', '.', c, c) and room != ('.', '.', '.', c) and room != ('.', '.', '.', '.') and hall[door] == '.':
        if room[0] != '.':
            for i in range(door - 1, -1, -1):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[0]] * (1 + door - i)
                        new_hall = list(hall)
                        new_hall[i] = room[0]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', room[1], room[2], room[3])])
                else:
                    break
            for i in range(door + 1, 11):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[0]] * (1 + i - door)
                        new_hall = list(hall)
                        new_hall[i] = room[0]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', room[1], room[2], room[3])])
                else:
                    break
        if room[0] == '.' and room[1] != '.':
            for i in range(door - 1, -1, -1):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[1]] * (2 + door - i)
                        new_hall = list(hall)
                        new_hall[i] = room[1]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.', room[2], room[3]) ])
                else:
                    break
            for i in range(door + 1, 11):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[1]] * (2 + i - door)
                        new_hall = list(hall)
                        new_hall[i] = room[1]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.', room[2], room[3])])
                else:
                    break
        if room[0] == '.' and room[1] == '.' and room[2] != '.':
            for i in range(door - 1, -1, -1):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[2]] * (3 + door - i)
                        new_hall = list(hall)
                        new_hall[i] = room[2]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.', '.', room[3]) ])
                else:
                    break
            for i in range(door + 1, 11):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[2]] * (3 + i - door)
                        new_hall = list(hall)
                        new_hall[i] = room[2]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.', '.', room[3])])
                else:
                    break
        if room[0] == '.' and room[1] == '.' and room[2] == '.' and room[3] != '.':
            for i in range(door - 1, -1, -1):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[3]] * (4 + door - i)
                        new_hall = list(hall)
                        new_hall[i] = room[3]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.', '.', '.') ])
                else:
                    break
            for i in range(door + 1, 11):
                if hall[i] == '.':
                    if i not in (2, 4, 6, 8):
                        n_cost = costs[room[3]] * (4 + i - door)
                        new_hall = list(hall)
                        new_hall[i] = room[3]
                        new_hall = tuple(new_hall)
                        ans.append([n_cost, new_hall, ('.', '.', '.', '.')])
                else:
                    break
    return ans

def enter_room2(i, c, hall, room):
    # return (can_enter, cost, new_hall, new_room)
    door = doors[c]
    if not (room == ('.', '.', '.', '.') or room == ('.', '.', '.', c) or room == ('.', '.', c, c) or room == ('.', c, c, c)):
        return False, None, None, None
    if i < door:
        for j in range(i + 1, door + 1):
            if hall[j] != '.':
                return False, None, None, None
    else:
        for j in range(i - 1, door-1, -1):
            if hall[j] != '.':
                return False, None, None, None
    cost = (abs(i - door) + 1) * costs[c]
    if room == ('.', '.', '.', '.'):
        cost += 3 * costs[c]
        new_room = ('.', '.', '.', c)
    elif room == ('.', '.', '.', c):
        cost += 2 * costs[c]
        new_room = ('.', '.', c, c)
    elif room == ('.', '.', c, c):
        cost += costs[c]
        new_room = ('.', c, c, c)
    else:
        new_room = (c, c, c, c)
    new_hall = list(hall)
    new_hall[i] = '.'
    return True, cost, tuple(new_hall), new_room
    
def moves2(cost, state):
    ans = []
    hall, room1, room2, room3, room4 = state
    for out_cost, new_hall, new_room1 in leave_room2('A', room1, hall):
        n_state = (new_hall, new_room1, room2, room3, room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])
    
    for out_cost, new_hall, new_room2 in leave_room2('B', room2, hall):
        n_state = (new_hall, room1, new_room2, room3, room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])
    
    for out_cost, new_hall, new_room3 in leave_room2('C', room3, hall):
        n_state = (new_hall, room1, room2, new_room3, room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])

    for out_cost, new_hall, new_room4 in leave_room2('D', room4, hall):
        n_state = (new_hall, room1, room2, room3, new_room4)
        n_cost = out_cost + cost
        ans.append([n_cost, n_state])
    
    for i, c in enumerate(hall):
        if c != '.':
            if c == 'A':
                room = room1
            elif c == 'B':
                room = room2
            elif c == 'C':
                room = room3
            else:
                room = room4
            can_enter, enter_cost, new_hall, new_room = enter_room2(i, c, hall, room)
            if can_enter:
                n_cost = cost + enter_cost
                if c == 'A':
                    n_state = (new_hall, new_room, room2, room3, room4)
                elif c == 'B':
                    n_state = (new_hall, room1, new_room, room3, room4)
                elif c == 'C':
                    n_state = (new_hall, room1, room2, new_room, room4)
                else:
                    n_state = (new_hall, room1, room2, room3, new_room)
                ans.append([n_cost, n_state])
    return ans

In [50]:
part2('inputs/day23_test.txt')

44169

In [51]:
part2('inputs/day23.txt')

47258